In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
os.environ['RESGEN_USER'] = 'pete'
os.environ['RESGEN_PASSWORD'] = 'testtest'

In [6]:
import os
import resgen as rg

rgc = rg.connect(
  os.getenv('RESGEN_USER'),
  os.getenv('RESGEN_PASSWORD')
)

In [7]:
rgc.username

'pete'

## Illingworth et al. 2015

In [11]:
illingworth = rgc.find_or_create_project('Illingworth et al 2015', group='paper-data')

### Recreate Data

In [12]:
iw_dict = dict([
    (d.name, d) for d in rgc.find_datasets(project=illingworth)
])
iw_dict

{'Illingworth_et_al_e14_RING1B_ChIP.bw': R2BPsWan: Illingworth_et_al_e14_RING1B_ChIP.bw,
 'Illingworth_et_al_2015_H3K27me3_ChIP-seq.bigWig': EFrYq959: Illingworth_et_al_2015_H3K27me3_ChIP-seq.bigWig}

## Boyle et al. 2019

In [13]:
project = rgc.find_or_create_project('Boyle et al 2019', group='paper-data')

## Recreate data

In [16]:
tags = {
    "datatype": "matrix",
    "filetype": "cooler",
    "assembly": "mm9"
}

ds = project.sync_dataset(
    filepath='ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE134nnn/GSE134826/suppl/GSE134826_I53A.mm9.mapq_30.1000.mcool',
    **tags
)
ds = project.sync_dataset(
    filepath='ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE134nnn/GSE134826/suppl/GSE134826_KO.mm9.mapq_30.1000.mcool',
    **tags
)
ds = project.sync_dataset(
    filepath='ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE134nnn/GSE134826/suppl/GSE134826_WT.mm9.mapq_30.1000.mcool',
    **tags
)

In [17]:
datasets = rgc.find_datasets(project=project)
ds_dict = dict([(d.name, d) for d in datasets])
ds_dict

{'GSE134826_WT.mm9.mapq_30.1000.mcool': dJOzO_eM: GSE134826_WT.mm9.mapq_30.1000.mcool,
 'GSE134826_I53A.mm9.mapq_30.1000.mcool': O3YsqgiN: GSE134826_I53A.mm9.mapq_30.1000.mcool,
 'GSE134826_KO.mm9.mapq_30.1000.mcool': MP4esFAf: GSE134826_KO.mm9.mapq_30.1000.mcool}

In [18]:
chromsizes = rgc.find_datasets(
  datatype='chromsizes', assembly='mm9'
)[0]
chrominfo = rgc.get_chrominfo(chromsizes)

In [19]:
gene_annotations = rgc.find_datasets(
    datatype='gene-annotations', assembly='mm9'
)[0]

In [27]:
import higlass
from higlass.client import View, Track
from higlass.utils import hg_cmap

def plot(initialXDomain, height=130):    
    heatmap_options = {
        "height": height,
        "labelPosition": 'topLeft',
        "labelTextOpacity": 0.5,
        "maxZoom": 9,
        "valueScaleMin": 0.001,
        "valueScaleMax": 0.1
    }

    wt_track = ds_dict['GSE134826_WT.mm9.mapq_30.1000.mcool'].hg_track(
                    'horizontal-heatmap',
                    name="Ring1b WT",
                    **{
                        **heatmap_options,
                        "colorbarPosition": "topRight"
                    }
                )
    i53a_track = ds_dict['GSE134826_I53A.mm9.mapq_30.1000.mcool'].hg_track(
                    'horizontal-heatmap',
                    name="Ring1b I53A",
                    **heatmap_options
                )

    ko_track = ds_dict['GSE134826_KO.mm9.mapq_30.1000.mcool'].hg_track(
                    'horizontal-heatmap',
                    name="Ring1b KO",
                    **heatmap_options
                )
    
    iw_h3k27me = iw_dict['Illingworth_et_al_2015_H3K27me3_ChIP-seq.bigWig'].hg_track(
        'horizontal-bar',
        name='H3K27me3'
    )
    iw_ring1b = iw_dict['Illingworth_et_al_e14_RING1B_ChIP.bw'].hg_track(
        'horizontal-bar',
        name='RING1B'
    )    
    return higlass.display(
        [
            View([wt_track, i53a_track, ko_track,
                  iw_h3k27me, iw_ring1b,
                chromsizes.hg_track(),
                gene_annotations.hg_track(),
            ],
            initialXDomain=initialXDomain,
            width=6),
            View([
                Track('simple-svg', position='top', height=height),
                (i53a_track / wt_track).change_attributes(
                    options={
                        **heatmap_options,
                        'colorbarPosition': 'topRight',
                        'colorRange': hg_cmap('coolwarm'),
                        'name': 'I53A / WT',
                        'valueScaleMin': 1 / 5,
                        'valueScaleMax': 5,
                        "scaleStartPercent": "0.00000",
                        "scaleEndPercent": "1"
                    }
                  ),
                  (ko_track / wt_track).change_attributes(
                    options={
                        **heatmap_options,
                        'colorRange': hg_cmap('coolwarm'),
                        'name': 'KO / WT',
                        'valueScaleMin': 1 / 5,
                        'valueScaleMax': 5,
                        "scaleStartPercent": "0.00000",
                        "scaleEndPercent": "1"
                    }
                  ),
            ],
                initialXDomain=initialXDomain, x=6, width=6, y=6)
        ]
    )
display,server,viewconf = plot(
    [
        chrominfo.to_abs('chr6', 51.8e6),
        chrominfo.to_abs('chr6', 52.55e6)
    ]
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'V_05pxlWQ72b9BgPAhDO-A', 'tracks': {'top': [{'ty…

In [28]:
project.sync_viewconf(viewconf.to_dict(), 'Figure 3A')

In [22]:
display,server,viewconf = plot(
    [
        chrominfo.to_abs('chr11', 95.75e6),
        chrominfo.to_abs('chr11', 96.5e6)
    ]
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'X4uMbNZITQ6GXuL1oZ_tjQ', 'tracks': {'top': [{'ty…

In [31]:
project.sync_viewconf(viewconf.to_dict(), 'Figure 3B')

In [32]:
display,server,viewconf = plot(
    [
        chrominfo.to_abs('chr11', 118.86e6),
        chrominfo.to_abs('chr11', 119.0e6)
    ]
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'QzhqVSOlSBudNVdGiMPwww', 'tracks': {'top': [{'ty…

In [33]:
project.sync_viewconf(viewconf.to_dict(), 'Figure 3C')

In [34]:
display,server,viewconf = plot(
    [
        chrominfo.to_abs('chr2', 17.7e6),
        chrominfo.to_abs('chr2', 18.8e6)
    ],
    height=150
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'ZB7t-_N2R6SOil13ujewrg', 'tracks': {'top': [{'ty…

In [35]:
project.sync_viewconf(viewconf.to_dict(), 'Figure 4A')